# Wrangling Project

### First of all importing  important packages 

In [1]:
import pandas as pd
import requests
import os
import collections
import json
import glob
import numpy as np
import re


## wrangling include this 3 steps 
- Gathering 
- Assessing 
- Cleaning

## Gather
### we have got 3 kinds of data to gather to gather in 3 ways 
- twitter_archive.csv
- image_prediction.tsv 
- twitter APIs in json formated .txt  

##### First is twitter_archive.csv

In [2]:
# this data part in on hand twitter_archive.csv
twitter_archive = pd.read_csv('twitter-archive-enhanced-2.csv')

In [3]:
#Exploer twitter_archive
twitter_archive.head(4)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None


##### Second is image_prediction 

In [4]:
# check if the folder with same name doesnt exist and if yes make a new folder named (image_predictions)
folder_name = 'image_predictions'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [ ]:
# get the image_prediction file from udacity server
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response

response [200] means the request has succeeded 


In [ ]:
# now we will write image prediction data to a file in our folder image_predictions
with open(os.path.join(folder_name, url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

os.listdir(folder_name)
image_predictions_df = pd.read_csv('image_predictions/image-predictions.tsv', sep='\t')

In [ ]:
# Exploer df_image_prediction
image_predictions_df.head(4)

##### Third is twitter API  

In [ ]:
# # Authentication part to get the data from twitter API 
# consumer_key = 'HIDDEN'
# consumer_secret = 'HIDDEN'
# access_token = 'HIDDEN'
# access_secret = 'HIDDEN'

# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)

# api = tweepy.API(auth, wait_on_rate_limit=True)

# # that part is tweets id's allow us to get the tweets infos from twitter API

# tweet_ids = twitter_archive.tweet_id.values
# len(tweet_ids)

# # Query Twitter's API for JSON data for each tweet ID in the Twitter archive
# count = 0
# fails_dict = {} # fail id's will be saved here (the id's which tweets either deleted or accounts were removed)
# start = timer()

# # Save each tweet's returned JSON as a new line in a .txt file
# with open('tweet_json.txt', 'w') as json_file:
#     # This loop will likely take 20-30 minutes to run because of Twitter's rate limit (it will get tweets infos from the api and if TweepError exception happend it wont break the loop cuz except statement then the fail id will be saved in the dict above)
#     for tweet_id in tweet_ids:
#         count += 1
#         print(str(count) + ": " + str(tweet_id))
#         try:
#             tweet = api.get_status(tweet_id, tweet_mode='extended')
#             print("Success")
#             json.dump(tweet._json, json_file) # to write the data which from twitter api in the txt file
#             json_file.write('\n')
#         except tweepy.TweepError as e:
#             print("Fail")
#             fails_dict[tweet_id] = e
#             pass
# end = timer()
# print(end - start)
# print(fails_dict)



In [ ]:
# tweet_status = pd.read_json(r'we_rate_dogs\tweet-json', lines = True,encoding='utf-8')

tweets_data = []
with open('tweet-json') as tweet_status:        
    for json_obj in tweet_status:
        df_tweet_min = collections.OrderedDict()
        df_tweet_full = json.loads(json_obj)

        # Get all of the data we're interested in
        df_tweet_min['tweet_id'] = df_tweet_full['id']
        df_tweet_min['retweet_count'] = df_tweet_full['retweet_count']
        df_tweet_min['favorite_count'] = df_tweet_full['favorite_count']

        # Append it to the data gathering list
        tweets_data.append(df_tweet_min)

# Create a dataframe from the data
tweet_api_df = pd.DataFrame(tweets_data)
tweet_api_df

##### The data has now been loaded into the following DFs:

- twitter_archive.csv as twitter_archive
- image_prediction.tsv as df_image_prediction
- twitter APIs in json formated (txt) as tweet_api_df

## Visual Assessing 
### Archive DF
i used excel to do this step
- theres none values as string not NAN so validation issue .
- source column has html tags which i think they aren't important and device column should show me the device and applicaton used if it web , vine or phone app (html tags meant to download the app for each device type) .
- timestamp has 00:00 which is duplicated and not necessary should be deleted .
- denominator and numerator columns has outliers (this one need to be checked with programmatically assessing to make sure) .


## Programmatic Assessing
### Archive
- (in_reply_to_status_id) and (in_reply_to_user_id) columns have alot of missing data only (78) and not needed data .
- before dropping them we need to drop the not null values to make sure our data cleaned from replies
- (retweeted_status_id), (retweeted_status_user_id), (retweeted_status_timestamp) columns have alot of missing data only (181) also not needed data . 
- before dropping them we need to drop these tweets to make sure our data cleaned from replies and 

- Timestamp column with wrong data type it should be Datetime not object
- demnoerator and numerator has different values from text i should change them manually. 
- many strange names for dogs in column 'name' like 'none', 'a', 'actually', 'O' and 'all' should be removed .
### Image predictions file 
- we have 2075 tweets with images while we archive has 2356 that means we have 281 rows with tweet_id that either retweets or somthing else than dogs .
- in df_image_prediction the name of columns (p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog) as p1, p2 and p3 contain the same type of data, predictions. aren't usefull to be used for analysis or understanding the data .


#### tidiness issues 
- there are 4 columns for dogs types which is tidy issue so we can make one column to get them all but when i scanned the data visually with excel i saw that there are two or more types for same dog which we can separate with commas.
- it would be easier for analysis after cleaning each Df make one table that has the three dfs (twitter_archive),(tweet_api_df), (df_image_prediction).
- as tidy data rules (p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog) not qualified as the columns p1, p2 and p3 contain the same type of data, predictions, The columns p1_conf, p2_conf and p3_conf all contain values for confidence level, and columns p1_dog, p2_dog and p3_dog all contain Boolean values indicating whether the prediction is in fact a type of dog, so i will melt them into 2 columns that tells us breed and confidence of the machine . 

##### those notes was taken after doing the assessing below 

In [ ]:
twitter_archive.info()

In [ ]:
twitter_archive[twitter_archive['name'].isnull()]

we got a problem here as all null values written as none (string) in name , and dog types columns

In [ ]:
tweet_api_df.info()

In [ ]:
image_predictions_df.info()

In [ ]:
# check if there dulpicates in DFs
sum(tweet_api_df.duplicated()),sum(twitter_archive.duplicated()),sum(image_predictions_df.duplicated())

In [ ]:
# check if there duplicates in tweet_id columns
sum(tweet_api_df.duplicated('tweet_id')),sum(twitter_archive.duplicated('tweet_id')),sum(image_predictions_df.duplicated('tweet_id'))


In [ ]:
all_columns = pd.Series(list(twitter_archive) + list(tweet_api_df) + list(image_predictions_df))
all_columns[all_columns.duplicated()]

In [ ]:
twitter_archive['rating_numerator'].value_counts()


In [ ]:
twitter_archive['rating_denominator'].value_counts()


In [ ]:
twitter_archive['name'].value_counts().index.tolist()

In [ ]:
twitter_archive.value_counts('name')['actually'],twitter_archive.value_counts('name')['all'],twitter_archive.value_counts('name')['a'],twitter_archive.value_counts('name')['O']

In [ ]:
twitter_archive.query('name == "a"')


In [ ]:
twitter_archive.query('rating_numerator == "0"')

In [ ]:
twitter_archive.query('rating_denominator == "0"')


In [ ]:
image_predictions_df.query('tweet_id == "835152434251116546"')

In [ ]:
image_predictions_df.query('tweet_id == "746906459439529985"')

not a dog

In [ ]:
image_predictions_df.query('tweet_id == "835246439529840640"')

In [ ]:
twitter_archive['text'][313]


In [ ]:
image_predictions_df.info()

In [ ]:
image_predictions_df.describe()

In [ ]:
twitter_archive.describe()

In [ ]:
twitter_archive.query('rating_numerator == "1776"')

In [ ]:
twitter_archive['text'][979]

he might love his dog so much 

In [ ]:
twitter_archive.query('rating_denominator == "170"')

In [ ]:
twitter_archive['text'][1120]

strange rating but still ok cuz that page is famous for it   

#### image prediction columns 
- p1 is the algorithm's #1 prediction for the image in the tweet 
- p1_conf is how confident the algorithm is in its #1 prediction 
- p1_dog is whether or not the #1 prediction is a breed of dog
- p2 is the algorithm's second most likely prediction 
- p2_conf is how confident the algorithm is in its #2 prediction
- p2_dog is whether or not the #2 prediction is a breed of dog etc.

In [ ]:
image_predictions_df.describe()

In [ ]:
image_predictions_df.query('p1_conf == "1.000000"')

In [ ]:
tweet_api_df.describe()

In [ ]:
tweet_api_df.query('retweet_count == "79515"')

highest tweet got retweets 

In [ ]:
twitter_archive.query('tweet_id == "744234799360020481"')

In [ ]:
twitter_archive['text'][1039]

In [ ]:
image_predictions_df.query('tweet_id == "744234799360020481"')

In [ ]:
image_predictions_df['jpg_url'][1221]

In [ ]:
tweet_api_df.query('retweet_count == "624"')

In [ ]:
image_predictions_df.query('tweet_id == "681242418453299201"')

the dog havent been recognized by the algorithm.

de

In [ ]:
twitter_archive[twitter_archive['rating_denominator'] > 11].text

In [ ]:
twitter_archive[twitter_archive['rating_numerator'] > 11].text

In [ ]:
twitter_archive['text'][320]

In [ ]:
twitter_archive['text'][433]

In [ ]:
twitter_archive['text'][902]

In [ ]:
twitter_archive['text'][1120]

In [ ]:
twitter_archive['text'][1165]

In [ ]:
twitter_archive['text'][1202]

In [ ]:
twitter_archive['text'][1228]

In [ ]:
twitter_archive['text'][1254]

In [ ]:
twitter_archive['text'][1274]

In [ ]:
twitter_archive['text'][1351]

In [ ]:
twitter_archive['text'][1433]

In [ ]:
twitter_archive['text'][1598]

In [ ]:
twitter_archive['text'][1634]

In [ ]:
twitter_archive['text'][1663]

In [ ]:
twitter_archive['text'][1779]

In [ ]:
twitter_archive['text'][1843]

###### to make sure that all  are tweets and with images i will do another assessing after cleaning .

#### summery 
    Quality
-  change 'None values' in name and dog types columns to empty string. 
-  remove html tags from source column.
-  change Timestamp column's type to datetime.
-  change the type of tweet_id:object, source: category, rating_numerator:float, retweet_count and favorite_count: int32 which would make it smaller as we wont use the 64 bit at all , dog_types:category
-  drop (in_reply_to_status_id) and (in_reply_to_user_id) columns
           before that we should drop not null values 
-  drop (retweeted_status_id), (retweeted_status_user_id), (retweeted_status_timestamp) columns
           before that we should drop not null values 
-  change the columns names in image_predictions_df. 
-  drop duplicated columns. 
-  drop tweets arent having images.   
-  drop null values from expanded_urls. 
-  change lower case (not names) from name column like 'a', 'actually', 'O' and 'all' to empty string and None values as well.
- last step would be change the names of columns for the final biggest df which will be combination of the 3 dfs , that would make my analysis easier .
    
    tidiness
-  append all 3 tables in one.
-  make one column (dog_types) by appending the 4 columns (doggo) (floofer) (pupper) (puppo).
-  The json_data table should be combined with the archive table.
-  change all p,p_conf,p_dog into 2 columns storing breed and confidence of the breed then drop these columns (p,p_conf,p_dog) .
-  restructure the columns which will help me in analysis .



## cleaning

In [ ]:
# Creat a copy of the original 3 datasets for reference
archive_clean_df = twitter_archive.copy()
image_clean_df = image_predictions_df.copy()
json_clean_df = tweet_api_df.copy()
archive_clean_df.info()

# 1
### define
starting with cleaning the unnecessary data in (in_reply_to_status_id) and (in_reply_to_user_id) columns as we dont need replies for analysis .

#### code 

In [ ]:
archive_clean_df = archive_clean_df[archive_clean_df.in_reply_to_user_id.isna()]

#### test

In [ ]:
archive_clean_df.info()

# 2
### define 
drop these empty columns will make my df better for analysis .

#### code

In [ ]:
archive_clean_df = archive_clean_df.drop(['in_reply_to_status_id','in_reply_to_user_id'], axis = 1)

#### test

In [ ]:
archive_clean_df.info()

# 3
### define 
we dont need any data about retweets (retweeted_status_id), (retweeted_status_user_id), (retweeted_status_timestamp).

#### code

In [ ]:
archive_clean_df = archive_clean_df[archive_clean_df.retweeted_status_id.isna()]

#### test

In [ ]:
archive_clean_df.info()

# 4
### Define
dropping these columns will make our df more tidy and less noisy when accessing data with or assessing again .

#### Code

In [ ]:
archive_clean_df = archive_clean_df.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'], axis = 1)

#### Test

In [ ]:
archive_clean_df.info()

now i have deleted rows that related to retweets and replies which is not necessary and all we have now is tweets df .

# 5
### define
before metling the 4 dog types into one column we need to change none values into ''

#### Code

In [ ]:
archive_clean_df['doggo'].replace(to_replace = 'None' ,value =  '', inplace=True),
archive_clean_df['floofer'].replace(to_replace = 'None' ,value =  '', inplace=True),
archive_clean_df['pupper'].replace(to_replace = 'None' ,value =  '', inplace=True),
archive_clean_df['puppo'].replace(to_replace = 'None' ,value =  '', inplace=True)

#### Test

In [ ]:
archive_clean_df.value_counts(['doggo','floofer','pupper','puppo'])

In [ ]:
archive_clean_df.info()

i made it like that to be able to convert the all four columns easily and i saw while assessing that some dogs have 2 or more types that would prevent losing these compound types.

# 6
### define
now for more tidyness i need to melt these columns ('doggo','floofer','puppo','pupper')  into one which is dog_types.

#### Code

In [ ]:
archive_clean_df['dog_types'] = archive_clean_df['doggo'] + archive_clean_df['floofer'] + archive_clean_df['pupper'] + archive_clean_df['puppo'] 

#### Test

In [ ]:
archive_clean_df.info()

In [ ]:
archive_clean_df['expanded_urls'].isnull().values.any()

In [ ]:
archive_clean_df['expanded_urls'].isnull().sum()

# 7
### Define
delete null values in expanded_urls column .

#### Code

In [ ]:
archive_clean_df = archive_clean_df[archive_clean_df.expanded_urls.notnull()]


#### Test

In [ ]:
archive_clean_df.info()

# 8
### Define
for tidyness dropping 4 dog types columns

#### Code

In [ ]:
archive_clean_df.drop(['doggo', 'pupper', 'floofer', 'puppo'], axis = 1 , inplace = True)
archive_clean_df.sample(10)

# 9
### Define
now merging archive_clean_df, json_clean_df .

#### Code

In [ ]:
df = pd.merge(archive_clean_df,json_clean_df)

#### Test

In [ ]:
json_clean_df.info()

In [ ]:
archive_clean_df.info()

In [ ]:
df.info()

# 10
### Define
we can clear the timestamp from +00:00

#### Code

In [ ]:
df.timestamp = df.timestamp.str.strip('+0000')

#### Test 

In [ ]:
df['timestamp'].sample(20)

# 11
### Define
Change timestamp type from object to datetime

#### Code

In [ ]:
df['timestamp'] = pd.to_datetime(df.timestamp)

#### Test

In [ ]:
df['timestamp'].dtypes

# 12
### Define
delete html tags and make it refering to the source of the tweet like phone or web.

### Code

In [ ]:
df['source'] = df['source'].str.extract('^<a.+>(.+)</a>$')

### Test

In [ ]:
df['source'].sample(20)

In [ ]:
df.source.value_counts()

# 13
### Define
while i was assessing vitually i saw that the strange names for dogs are always .islower (small_character_name) so i will change it to '' like what i have done in dog_types .

#### Code

In [ ]:
small_character_name = df.name.str.contains('^[a-z]')
df.loc[small_character_name, 'name'] = ''

#### Test

In [ ]:
df.name.value_counts()

# 14
### Define
change none values to make it '' like the rest of NAN values.

#### Code

In [ ]:
df.name.replace('None', '', inplace=True)

#### Test

In [ ]:
df.name.value_counts()

# 15
#### Define
in image_prediction_df i thought its not usefull to get all predictions as some are false and for every one has its confidence so i thought about making 2 columns getting in them the prediction with true value and its confidence .

#### Code

In [ ]:
# create 2 empty lists to save our choice for each row in the dataset
confidence = []
breed = []

# function that iterates through prediction columns to find the best prediction which is a breed of dog.
def Breeds_and_confidence(row):
    if row['p1_dog'] == True:
        breed.append(row['p1'])
        confidence.append(row['p1_conf'])
    elif row['p2_dog'] == True:
        breed.append(row['p2'])
        confidence.append(row['p2_conf'])
    elif row['p3_dog'] == True:
        breed.append(row['p3'])
        confidence.append(row['p3_conf'])
    else:
        breed.append('Unknown')
        confidence.append(0)
        
# call function using pandas apply by columns
image_clean_df.apply(Breeds_and_confidence, axis=1)

# add lists created to master dataframe
image_clean_df['Breed'] = breed
image_clean_df['Confidence'] = confidence

#### Test

In [ ]:
image_clean_df.info()

now i took what i need for analysis in this df now i will drop the prediction columns and confidence

In [ ]:
image_clean_df.head()

# 16
### Define
now i took what i need for analysis in this df now i will drop the prediction columns and confidence

#### Code

In [ ]:
image_clean_df.drop(['p1','p1_conf','p2','p2_conf','p3','p3_conf','p1_dog','p2_dog','p3_dog','img_num'], axis = 1 , inplace = True)

#### Test

In [ ]:
image_clean_df.sample(50)

In [ ]:
image_predictions_df.query('tweet_id == "783839966405230592"')

In [ ]:
image_predictions_df.query('tweet_id == "681242418453299201"')

In [ ]:
image_predictions_df.query('tweet_id == "695064344191721472"')

In [ ]:
image_predictions_df.query('tweet_id == "686749460672679938"')

well 'Unkown' breed and confidence 0 is not a valid data .

# 17
### Define
now we can merge all dfs in big one called df_master

In [ ]:
df.info()

In [ ]:
image_clean_df.info()

### Code

In [ ]:
df_all = pd.merge(df,image_clean_df)

### Test

In [ ]:
df_all.info()

that change because some tweets in archive didnt have images in image_prediction_df and there were images with tweet_ids are same for replies which was deleted from archive .

In [ ]:
df_all.head(5)

# 18
### Define
now lets change some types which would be more realistic .

#### Code

In [ ]:
df_all['tweet_id'] = df_all['tweet_id'].astype(object)
df_all['source'] = df_all['source'].astype('category')
df_all['rating_numerator'] = df_all['rating_numerator'].astype(float)
df_all['retweet_count'] = df_all['retweet_count'].astype(int)
df_all['favorite_count'] = df_all['favorite_count'].astype(int)
df_all['dog_types'] = df_all['dog_types'].astype('category')
df_all['Confidence'] = df_all['Confidence'].round(2) # after cleaning and assessing again i can see that its numbers too long 

#### Test

In [ ]:
df_all.info()

In [ ]:
df_all.head(5)

# 19
### Define 
while assessing i found a few data in rating_denominator, rating_numerator that has different value from what in text .

In [ ]:
df_all[df_all['rating_denominator'] > 11].text

##### Note. if i saw rating in the text i will change the rating to it if not i will calculate the rating from the exiting data.

##### 1

In [ ]:
df_all['text'][320]

In [ ]:
df_all['text'][658]

In [ ]:
df_all['text'][842]

In [ ]:
df_all['text'][884]

In [ ]:
df_all['tweet_id'][320],df_all['tweet_id'][658],df_all['tweet_id'][842],df_all['tweet_id'][884]

In [ ]:
df_all['rating_numerator'][320],df_all['rating_numerator'][658],df_all['rating_numerator'][842],df_all['rating_numerator'][884]

In [ ]:
df_all['rating_denominator'][320],df_all['rating_denominator'][658],df_all['rating_denominator'][842],df_all['rating_denominator'][884]

In [ ]:
# for each tweet id above
df_all.loc[df_all.tweet_id == 820690176645140481, 'rating_numerator'] = 12 #84/70= 1.2*10 = 12
df_all.loc[df_all.tweet_id == 820690176645140481, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 758467244762497024, 'rating_numerator'] = 11 #165/150
df_all.loc[df_all.tweet_id == 758467244762497024, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 731156023742988288, 'rating_numerator'] = 12 #204/170= 1.2*10 = 12
df_all.loc[df_all.tweet_id == 731156023742988288, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 722974582966214656, 'rating_numerator'] = 13 # 4/20
df_all.loc[df_all.tweet_id == 722974582966214656, 'rating_denominator'] = 10

##### 2

In [ ]:
df_all['text'][918]

In [ ]:
df_all['text'][939]

In [ ]:
df_all['text'][963]

In [ ]:
df_all['text'][981]

In [ ]:
df_all['tweet_id'][918],df_all['tweet_id'][939],df_all['tweet_id'][963],df_all['tweet_id'][981]

In [ ]:
df_all['rating_numerator'][918],df_all['rating_numerator'][939],df_all['rating_numerator'][963],df_all['rating_numerator'][981]

In [ ]:
df_all['rating_denominator'][918],df_all['rating_denominator'][939],df_all['rating_denominator'][963],df_all['rating_denominator'][981]

In [ ]:
# for each tweet id above
df_all.loc[df_all.tweet_id == 716439118184652801, 'rating_numerator'] = 11 #50/50
df_all.loc[df_all.tweet_id == 716439118184652801, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 713900603437621249, 'rating_numerator'] = 11 #99/90
df_all.loc[df_all.tweet_id == 713900603437621249, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 710658690886586372, 'rating_numerator'] = 10 #80/80
df_all.loc[df_all.tweet_id == 710658690886586372, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 709198395643068416, 'rating_numerator'] = 9 # 45/50
df_all.loc[df_all.tweet_id == 709198395643068416, 'rating_denominator'] = 10

##### 3

In [ ]:
df_all['text'][1045]

In [ ]:
df_all['text'][1288]

In [ ]:
df_all['text'][1420]

In [ ]:
df_all['text'][1478]

In [ ]:
df_all['tweet_id'][1045],df_all['tweet_id'][1288],df_all['tweet_id'][1420],df_all['tweet_id'][1478]

In [ ]:
df_all['rating_numerator'][1045],df_all['rating_numerator'][1288],df_all['rating_numerator'][1420],df_all['rating_numerator'][1478]

In [ ]:
df_all['rating_denominator'][1045],df_all['rating_denominator'][1288],df_all['rating_denominator'][1420],df_all['rating_denominator'][1478]

In [ ]:
# for each tweet id above
df_all.loc[df_all.tweet_id == 704054845121142784, 'rating_numerator'] = 12 #60/50
df_all.loc[df_all.tweet_id == 704054845121142784, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 684222868335505415, 'rating_numerator'] = 11 #121/110
df_all.loc[df_all.tweet_id == 684222868335505415, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 677716515794329600, 'rating_numerator'] = 12 #144/120
df_all.loc[df_all.tweet_id == 677716515794329600, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 675853064436391936, 'rating_numerator'] = 11 # 88/80
df_all.loc[df_all.tweet_id == 675853064436391936, 'rating_denominator'] = 10

In [ ]:
df_all[df_all['rating_numerator'] > 14].text

In [ ]:
df_all['text'][382]

In [ ]:
df_all['rating_numerator'].describe()

as u can see theres no rating and i cant give anything for 24/7 so i will give it average rating of the df which is 11.5/10

In [ ]:
df_all['text'][499]

In [ ]:
df_all['text'][549]

In [ ]:
df_all['text'][722]

In [ ]:
df_all['text'][1120]

In [ ]:
df_all['text'][1359]

In [ ]:
df_all['text'][1696]

In [ ]:
df_all['tweet_id'][382],df_all['tweet_id'][499],df_all['tweet_id'][549],df_all['tweet_id'][722],df_all['tweet_id'][1120],df_all['tweet_id'][1359],df_all['tweet_id'][1696]

In [ ]:
df_all['rating_numerator'][382],df_all['rating_numerator'][499],df_all['rating_numerator'][549],df_all['rating_numerator'][722],df_all['rating_numerator'][1120],df_all['rating_numerator'][1359],df_all['rating_numerator'][1696]

In [ ]:
df_all['rating_denominator'][382],df_all['rating_denominator'][499],df_all['rating_denominator'][549],df_all['rating_denominator'][722],df_all['rating_denominator'][1120],df_all['rating_denominator'][1359],df_all['rating_denominator'][1696]

In [ ]:
# for each tweet id above
df_all.loc[df_all.tweet_id == 810984652412424192, 'rating_numerator'] = 11.5 #24/7
df_all.loc[df_all.tweet_id == 810984652412424192, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 786709082849828864, 'rating_numerator'] = 9.75 #75/10
df_all.loc[df_all.tweet_id == 786709082849828864, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 778027034220126208, 'rating_numerator'] = 11.27 #27/10
df_all.loc[df_all.tweet_id == 778027034220126208, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 749981277374128128, 'rating_numerator'] = 17.5 # 1776/10
df_all.loc[df_all.tweet_id == 749981277374128128, 'rating_denominator'] = 10

df_all.loc[df_all.tweet_id == 697463031882764288, 'rating_numerator'] = 11 #44/40
df_all.loc[df_all.tweet_id == 697463031882764288, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 680494726643068929, 'rating_numerator'] = 11.26 #26/10
df_all.loc[df_all.tweet_id == 680494726643068929, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 670842764863651840, 'rating_numerator'] = 14.2 #420/10
df_all.loc[df_all.tweet_id == 670842764863651840, 'rating_denominator'] = 10 

In [ ]:
df_all['rating_numerator'] = df_all['rating_numerator'].round(2)

making sure that doesnt exceed 2 digits 

In [ ]:
df_all.rating_numerator.value_counts()


In [ ]:
df_all[df_all['rating_denominator'] > 10].text

In [ ]:
df_all[df_all['rating_denominator'] < 10].text

In [ ]:
df_all['text'][794]

In [ ]:
df_all['text'][1313]

In [ ]:
df_all['text'][1950]

In [ ]:
df_all['tweet_id'][794],df_all['tweet_id'][1313],df_all['tweet_id'][1950]

In [ ]:
df_all['rating_numerator'][794],df_all['rating_numerator'][1313],df_all['rating_numerator'][1950]

In [ ]:
df_all['rating_denominator'][794],df_all['rating_denominator'][1313],df_all['rating_denominator'][1950]

In [ ]:
# for each tweet id above
df_all.loc[df_all.tweet_id == 740373189193256964, 'rating_numerator'] = 14 #9/11
df_all.loc[df_all.tweet_id == 740373189193256964, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 682962037429899265, 'rating_numerator'] = 10 #7/11 that meant be the store
df_all.loc[df_all.tweet_id == 682962037429899265, 'rating_denominator'] = 10 

df_all.loc[df_all.tweet_id == 666287406224695296, 'rating_numerator'] = 9 #1/2 it meant half breed 
df_all.loc[df_all.tweet_id == 666287406224695296, 'rating_denominator'] = 10 

# 20 
### Define
- last step would be changing the names of columns and restructure them to make my analysis easier.
- and replace breeds that are unknown .
- as we managed to make all denominators 10 so i think it would be ok to delete it which 'Out_of' .

#### Code

In [ ]:
# change their names 
df_all = df_all.rename(columns = { 'tweet_id': 'Tweet_id',
                                   'timestamp':'Tweet_date',
                                   'source': 'Tweet_source',
                                   'text': 'Tweet_text',
                                   'expanded_urls': 'Tweet_url',
                                   'rating_numerator': 'Dog_score/10',
                                   'rating_denominator':'Out_of',
                                   'dog_types': 'Dog_type',
                                   'name': 'Dog_name',
                                   'favorite_count':'Num_of_likes',
                                   'retweet_count' :'Retweets',
                                   'jpg_url': 'Image_link',
                                   'Breed': 'Dog_Breed',
                                   'Confidence': 'Probability_AI_calc'})

In [ ]:
# change its structure to make the data needed for analysis easy to access 
df_all = df_all[['Tweet_id','Tweet_date','Tweet_source','Tweet_text','Dog_name',
                 'Dog_score/10','Out_of','Dog_type','Dog_Breed','Probability_AI_calc',
                 'Num_of_likes','Retweets','Tweet_url','Image_link']]

In [ ]:
df_all.Dog_Breed.replace('None', '', inplace=True)

In [ ]:
df_all.Dog_Breed.replace('Unknown', '', inplace=True)

In [ ]:
df_all.drop(['Out_of'], axis = 1 , inplace = True)

#### Test

In [ ]:
df_all.info()

In [ ]:
df_all.sample(20)

it seems to be fine for me now we can store it in twitter_archive_master.csv

# Storing

In [ ]:
df_all.to_csv('twitter_archive_master.csv', encoding='utf-8',index=False)


In [ ]:
df_all_cleaned = pd.read_csv('twitter_archive_master.csv')

In [ ]:
df_all_cleaned.info()

In [ ]:
df_all_cleaned.sample(10)

# Analysis

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_all_c = df_all_cleaned.copy()

In [ ]:
df_all_c.info()

In [ ]:
df_all_c.sample(50)

In [ ]:
df_all_c.describe()

starting with getting simple statistics 

In [ ]:
# tweets range in time
print('we have data since {} untill {}'.format(df_all_c.Tweet_date.min(),df_all_c.Tweet_date.max()))

##### i thought about getting a dog so i will try to get some information out of this data that could help me choose which breed to raise .

i like german shepherd so lets investigate 
### is there a german shephered? 

In [ ]:
df_all_c.query('Dog_Breed == "German_shepherd"')

In [ ]:
len(df_all_c.query('Dog_Breed == "German_shepherd"'))

not popular among this community only 21 tweets , its rating is ?

In [ ]:
df_all_c.query('Dog_Breed == "German_shepherd"')['Dog_score/10'].mean()

as u can see  theres 21 tweets about german sheperd i didnt expect that as i like this breed alot with avg rating 11/10 not bad

### Which breeds are most tweeted about? 

In [ ]:
df_all_c.Dog_Breed.value_counts().nlargest(20).sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(20,15))
df_all_c.Dog_Breed.value_counts().nlargest(20).sort_values(ascending=False).plot.pie()
plt.title("Top 20 breeds with most tweets ",fontsize=18)
plt.ylabel(" ")
plt.xlabel("");

In [ ]:
plt.figure(figsize=(12,8))
df_all_c.Dog_Breed.value_counts().nlargest(20).sort_values(ascending=False).plot.bar(color= 'black')
plt.title("Top 20 breeds with most tweets ",fontsize=18)
plt.ylabel("Average tweets per each breed ")
plt.xlabel("");

as u can see we have got golden_retriever first which means that this breed is popular among this community  

### Note .
if i used the the most likes , retweets or score (rating) for each breed without taking their tweets count it wouldnt be accurate so i should take the avg per each breed / per each tweet  
Example :  if we have 100000 likes for a breed with 106 tweets it would be 100000/106 = 943
and for 50000 likes for a breed with 20 tweets 50000/20 = 2500 which mean that second breed has bigger mean per tweet.

### what about retweets?

if i got each pack of breed tweets's retweets and divide it by the each breed tweets i will get the mean of every breed's tweet'retweets . 

In [ ]:
df_all_c.Dog_Breed.value_counts().nlargest(20)

In [ ]:
df_all_c['Retweets'].sum()

In [ ]:
df_all_c.groupby('Dog_Breed')['Retweets'].sum().nlargest(20)

In [ ]:
num_of_ret_per_breed = df_all_c.groupby('Dog_Breed')['Retweets'].mean().nlargest(20)

In [ ]:
df_all_c.groupby('Dog_Breed')['Retweets'].mean().nlargest(20)

In [ ]:
plt.figure(figsize=(12,8))
num_of_ret_per_breed.nlargest(20).plot.bar(color= 'black')
plt.title("Top 20 breeds with most retweets ",fontsize=18)
plt.ylabel("Average retweets per breed ")
plt.xlabel("");

Bedlington_terrier in first place , now w've got golden_retriever as popularity among dog's ownsers in this community in that time era and Bedlington_terrier with  most avg retweets per breed .

### Which breed has most favourite? 


In [ ]:
df_all_c['Num_of_likes'].sum()

In [ ]:
df_all_c.groupby('Dog_Breed')['Num_of_likes'].sum().nlargest(20)

In [ ]:
num_of_fav_per_breed = df_all_c.groupby('Dog_Breed')['Num_of_likes'].mean().nlargest(20)

In [ ]:
df_all_c.groupby('Dog_Breed')['Num_of_likes'].mean().nlargest(20)

In [ ]:
plt.figure(figsize=(12,8))
num_of_fav_per_breed.nlargest(20).plot.bar(color= 'black')
plt.title("Top 20 breeds with most favourite per tweet ",fontsize=18)
plt.ylabel("Average favourite per breed ")
plt.xlabel("");

Saluki in the first place , it becomes harder to get a decision about which better breed .
### rating per each breed 

In [ ]:
df_all_c['Dog_score/10'].sum()

In [ ]:
df_all_c.groupby('Dog_Breed')['Dog_score/10'].sum().nlargest(20)

In [ ]:
avg_score_per_breed = df_all_c.groupby('Dog_Breed')['Dog_score/10'].mean().nlargest(20)

In [ ]:
df_all_c.groupby('Dog_Breed')['Dog_score/10'].mean().nlargest(20)

In [ ]:
plt.figure(figsize=(12,8))
avg_score_per_breed.nlargest(20).plot.barh(color= 'black')
plt.title("Top 20 breeds with most rating per tweet ",fontsize=18)
plt.ylabel("Average rating per breed ")
plt.xlabel("");

i thought about a question that doesnt do anything with making decision about getting a dog 
and this que is, 
### which breed the machine recognized better ? 
60% to 100%

In [ ]:
df_all_c.groupby('Dog_Breed')['Probability_AI_calc'].mean().nlargest(20)

In [ ]:
avg_probability_per_breed=df_all_c.groupby('Dog_Breed')['Probability_AI_calc'].mean()

In [ ]:
plt.figure(figsize=(20,15))
avg_probability_per_breed.nlargest(20).plot.bar(color= 'black')
plt.title("Top 20 breeds with highest probability being recognized from the machine ",fontsize=18)
plt.ylabel("Average probability per breed ")
plt.xlabel("");

### which dog had 100% recognized by the machine ?

In [ ]:
df_all_c.query('Probability_AI_calc == 1')

In [ ]:
mask = df_all_c.query('Probability_AI_calc == 1')

In [ ]:
mask.groupby('Dog_Breed')['Probability_AI_calc'].count().nlargest(20)

pug has got 8 times , this dog look is special 

### is there a relation between favourite and retweet ? and what is that relation?


In [ ]:
plt.figure(figsize=(20, 15))
df_all_c.plot.scatter(x='Retweets',
                      y='Num_of_likes');

its positive correlation and with some outliers  .

## Sources
- udacity class room notes .
- notes from my doctors in university and my colleagues.
- https://pandas.pydata.org/
- https://stackoverflow.com/
- https://nfpdiscussions.udacity.com/   